In [1]:
import os
os.chdir("../")
%pwd

'd:\\DataScience\\Thyroid-disease-prediction'

In [63]:
# Preparing Data Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir : Path
    SATUTS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [64]:
#preparing Configuration
from Thyroid_Disease.constants import *
from Thyroid_Disease.utils.common import read_yaml,create_directories

In [70]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        data_validtion_config = DataValidationConfig(
            root_dir=config.root_dir,
            SATUTS_FILE= config.STATUS_FILE,
            unzip_data_dir= config.unzip_data_dir,
            all_schema = schema,
        )
        create_directories([config.root_dir])

        return data_validtion_config


In [66]:
# Ceating Components
import os
from Thyroid_Disease import logger
import pandas as pd

In [75]:
class DataValidation:
    def __init__(self, config: DataValidationConfig): self.config = config
    
    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                else:
                    validation_status = True

                with open(self.config.SATUTS_FILE, "w") as f:
                        f.write(f"Validation status: {validation_status}")
                         
            return validation_status
        except Exception as e:
            raise e

In [76]:
# pipeline

try:
  config = ConfigurationManager()
  data_validation_config = config.get_data_validation_config()
  data_validation =DataValidation(data_validation_config)  # Pass the config object as an argument
  data_validation.validate_all_columns()
except Exception as e:
  raise e


[2024-04-02 07:20:52,972 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-04-02 07:20:52,974 : INFO : common : yaml file: params.yaml loaded successfully]
[2024-04-02 07:20:52,989 : INFO : common : yaml file: schema.yaml loaded successfully]
[2024-04-02 07:20:52,991 : INFO : common : created directory at: artifacts]
[2024-04-02 07:20:52,994 : INFO : common : created directory at: artifacts/data_validation]
